# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [45]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Week 1: Introduction

No code contribution expected this week

# Week 2: Camera System Modeling and Operations

We plan to
- Model the simple pinhole camera system
- Write utility functions to
    - project a 3D world point to an image
    - Compute image footprint on a surface
    - Compute the Ground Sampling Distance

## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [ ]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20856347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [47]:
print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x=8192, image_size_y=6144)


## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [48]:

point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [49]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831271 124.46090238]


In [50]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662541 248.92180476]


## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

Note: Please return just one value of the GSD. Take the mininum of the values along the two axes.

In [51]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020257308330412907


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are consistent.

# Week 3: Model the user requirements

For this week, we will model the dataset specifications.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned
- Scan_dimension_y: the vertical size of the rectangle to be scanned
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [52]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


# Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [53]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.16791291 11.38070491]


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


### $\color{Cyan}{\text{Camera Parameters Experiments}}$

Experiment 1: Double the flight height in dataset_spec_.
* **Prediction:** Doubling the flight height will double both distance_x (distance along the flight path) and distance_y (distance between flight lines)

* **Reasoning:**
    * The image footprint (footprint_x, footprint_y) is linearly proportional to the distance_from_surface (height), as seen in the compute_image_footprint_on_surface calculation: footprint = (sensor_size * distance) / focal_length_mm.

In [54]:
# Experiment 1 - Double the flight height
print("Experiment 1: Double Height")
camera_1 = copy.copy(camera_x10)
dataset_spec_1 = copy.copy(dataset_spec)
original_height = dataset_spec_1.height
dataset_spec_1.height = original_height * 2 # Double the height
computed_distances_1 = compute_distance_between_images(camera_1, dataset_spec_1)
print(f"Computed distance (Height x2): {computed_distances_1}")
print(f"Original distance: {computed_distances}\n" + "="*30 + "\n")

Experiment 1: Double Height
Computed distance (Height x2): [30.33582583 22.76140983]
Original distance: [15.16791291 11.38070491]



Experiment 2: Increase Overlap Ratio
* **Prediction:** Increasing the `overlap` ratio will *decrease* `distance_x` while `distance_y` will stay the same.
* **Reasoning:**
    *   The image footprint width (`footprint_x`) remains constant because the `height` and camera parameters are unchanged.
    *   As the `overlap` ratio increases: $$ (1 - \text{overlap})$$ this value decreases. This means the drone needs to travel a shorter distance forward between photos to achieve the required higher overlap.

In [55]:
# Experiment 2 - Increase the overlap ratio
print("Experiment 2: Increase Overlap")
camera_2 = copy.copy(camera_x10)
dataset_spec_2 = copy.copy(dataset_spec)
dataset_spec_2.overlap = 0.9 # Increase overlap to 90%
computed_distances_2 = compute_distance_between_images(camera_2, dataset_spec_2)
print(f"Computed distance (Overlap = 0.9): {computed_distances_2}")
print(f"Original distance: {computed_distances}\n" + "="*30 + "\n")

Experiment 2: Increase Overlap
Computed distance (Overlap = 0.9): [ 5.05597097 11.38070491]
Original distance: [15.16791291 11.38070491]



Experiment 3: Increase Sidelap Ratio
* **Prediction:**  Increasing the `sidelap` ratio will *decrease* `distance_y` and `distance_x` will stay the same.
* **Reasoning:**
    *   As the `sidelap` ratio increases: $$ (1 - \text{sidelap})$$ this value decreases. This means the adjacent flight lines need to be closer together (smaller `distance_y`) to achieve the required higher sidelap.

In [56]:

# Experiment 3 - Increase the sidelap ratio
print("Experiment 3: Increase Sidelap")
camera_3 = copy.copy(camera_x10)
dataset_spec_3 = copy.copy(dataset_spec)
dataset_spec_3.sidelap = 0.9 # Increase sidelap to 90%
computed_distances_3 = compute_distance_between_images(camera_3, dataset_spec_3)
print(f"Computed distance (Sidelap=0.9): {computed_distances_3}")
print(f"Original distance: {computed_distances}\n" + "="*30 + "\n")

Experiment 3: Increase Sidelap
Computed distance (Sidelap=0.9): [15.16791291  3.7935683 ]
Original distance: [15.16791291 11.38070491]



Experiment 4: Increase Camera Focal Length (fx, fy)
* **Prediction:**  Increasing `fx` and `fy` (focal length in pixels) will *decrease* the image footprint. Consequently, both `distance_x` and `distance_y` will *decrease*.
* **Reasoning:**
    * `distance_x` and `distance_y` are directly proportional to `footprint_x` and `footprint_y`

In [57]:
# Experiment 4 - Increase Camera Focal Length
print("Experiment 4: Increase Focal Length (fx, fy)")
camera_4 = copy.copy(camera_x10)
dataset_spec_4 = copy.copy(dataset_spec)
camera_4.fx = camera_x10.fx * 1.5 # Increase fx by 50%
camera_4.fy = camera_x10.fy * 2 # Increase fy by 100%
computed_distances_4 = compute_distance_between_images(camera_4, dataset_spec_4)
print(f"Computed distance (Focal Length x1.5 & x2): {computed_distances_4}")
print(f"Original distance: {computed_distances}\n" + "="*30 + "\n")

Experiment 4: Increase Focal Length (fx, fy)
Computed distance (Focal Length x1.5 & x2): [10.11194194  5.69035246]
Original distance: [15.16791291 11.38070491]




Experiment 5: Increase Image Resolution (image_size_x, image_size_y)
* **Prediction:**  Increasing `image_size_x` and `image_size_y` (while keeping `fx`/`fy` and `sensor_size_mm` constant in the `Camera` object) will *decrease* the calculated physical focal length (`focal_length_mm`). This will *increase* the image footprint (`footprint_x`, `footprint_y`). Consequently, both `distance_x` and `distance_y` will *increase*.
* **Reasoning:**
    * $$ \text{focal\_length\_mm} = \text{camera.fx} \times \frac{\text{sensor\_size\_mm}}{\text{image\_size\_px}} $$
        If `image_size_px` increases while `camera.fx` and `sensor_size_mm` remain constant, the calculated `focal_length_mm` *decreases*.
    *   $$ \text{footprint} = \frac{\text{sensor\_size\_mm} \times \text{distance\_mm}}{\text{focal\_length\_mm}} $$
        A smaller `focal_length_mm` in the denominator results in a *larger* calculated footprint.
    *  Since `distance_x` and `distance_y` are directly proportional to `footprint_x` and `footprint_y` they will also increase.

In [58]:

# Experiment 5 - Increase Sensor Size
print("Experiment 5: Increase Image Resolution")
camera_5 = copy.copy(camera_x10)
dataset_spec_5 = copy.copy(dataset_spec)
camera_5.image_size_x = (camera_x10.image_size_x * 1.5) # Increase resolution by 50%
camera_5.image_size_y = (camera_x10.image_size_y * 1.5) # Increase resolution by 50%
computed_distances_5 = compute_distance_between_images(camera_5, dataset_spec_5)
print(f"Computed distance (Resolution x1.5): {computed_distances_5}")
print(f"Original distance: {computed_distances}\n" + "="*30 + "\n")

Experiment 5: Increase Image Resolution
Computed distance (Resolution x1.5): [22.75186937 17.07105737]
Original distance: [15.16791291 11.38070491]



## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [59]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.09


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


### $\color{Cyan}{\text{Maximum Speed Experiments}}$

#### Experiment 1: Increase Flight Height

*   **Parameter Change:** Double the flight `height`
*   **Prediction:** Doubling the flight height will *increase* the maximum allowed speed during photo capture.
*   **Reasoning:**
    *   Ground Sampling Distance increases linearly with height (`distance_from_surface`). A larger GSD means that 1 pixel of allowed movement corresponds to a larger physical distance on the ground.
    *   Since the allowed ground distance increases with height the maximum permissible speed increases.
*   **Importance in the Field:** This demonstrates the relationship between flight altitude and the speed constraints imposed by motion blur. While flying higher reduces ground resolution (larger GSD), it allows the drone to fly faster *during* image capture for the same pixel blur tolerance.

In [60]:
# Experiment 1 - Double the flight height
print("Experiment 1: Double Height")
camera_1 = copy.copy(camera_x10)
dataset_spec_1 = copy.copy(dataset_spec)
original_height = dataset_spec_1.height
dataset_spec_1.height = original_height * 2 # Double the height
print(f"Modified Parameters: height = {dataset_spec_1.height:.2f}m")
computed_speed_1 = compute_speed_during_photo_capture(camera_1, dataset_spec_1)
print(f"Original speed: {computed_speed:.2f} m/s")
print(f"Computed speed (Height x2): {computed_speed_1:.2f} m/s")

Experiment 1: Double Height
Modified Parameters: height = 60.96m
Original speed: 3.09 m/s
Computed speed (Height x2): 6.17 m/s


#### Experiment 2: Increase Exposure Time

*   **Parameter Change:** Increase the `exposure_time_ms`
*   **Prediction:** Increasing the exposure time will *decrease* the maximum allowed speed during photo capture.
*   **Reasoning:**
    *   As the `exposure_time_seconds` increases, the maximum permissible speed must decrease to ensure the drone doesn't cover more than the maximum allowed ground distance *while the shutter is open*.
*   **Importance in the Field:** This is critical for adapting flights to changing environmental conditions. In lower light, longer exposures are necessary, which directly forces a reduction in the drone's speed during capture to prevent motion blur.

In [61]:
print("Experiment 2: Increase Exposure Time")
camera_2 = copy.copy(camera_x10)
dataset_spec_2 = copy.copy(dataset_spec)
dataset_spec_2.exposure_time_ms = 4
computed_speed_2 = compute_speed_during_photo_capture(camera_2, dataset_spec_2)
print(f"New speed (Exposure = {dataset_spec_2.exposure_time_ms}ms): {computed_speed_2:.2f} m/s")
print(f"Original speed (Exposure = {exposure_time_ms}ms): {computed_speed:.2f} m/s")


Experiment 2: Increase Exposure Time
New speed (Exposure = 4ms): 1.54 m/s
Original speed (Exposure = 2ms): 3.09 m/s


# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [62]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 165 waypoints


In [63]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:20]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

Idx 0: Waypoint(x=0.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 1: Waypoint(x=15.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 2: Waypoint(x=30.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 3: Waypoint(x=45.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 4: Waypoint(x=60.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 5: Waypoint(x=75.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 6: Waypoint(x=90.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 7: Waypoint(x=105.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 8: Waypoint(x=120.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 9: Waypoint(x=135.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 10: Waypoint(x=150.0, y=0.0, z=30.48, speed=3.0872137895549265)
Idx 11: Waypoint(x=150.0, y=10.714285714285714, z=30.48, speed=3.0872137895549265)
Idx 12: Waypoint(x=135.0, y=10.714285714285714, z=30.48, speed=3.0872137895549265)
Idx 13: Waypoint(x=120.0, y=10.714285714285714, z=30.48, speed=3.0872137895549265)
Idx 14: Waypoint(x=10

## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal speed profile

# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [64]:
fig = plot_photo_plan(computed_plan)
print(f"Generated {len(plan_abl4)} waypoints.")
fig.show()

Generated 165 waypoints.


$\color{red}{\text{TODO: }}$ Compute the following ablations (and any other you can think of). 
You need to describe the input params you are changing, what impact you can observe, explanation behind the change in output, and practical implication of the correlation.

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

### <font color='dodgerblue'>Ablation 1: Increase Overlap</font>

*   **Change:** Increase `dataset_spec.overlap` from the baseline (0.7) to 0.9.
*   **Observation:** The waypoints along each horizontal flight line are now much closer together, increasing the total number of waypoints significantly. The vertical spacing between flight lines remains unchanged.
*   **Reasoning:** Requiring more overlap between consecutive photos means the drone must travel a shorter distance forward before taking the next picture. Since the sidelap requirement didn't change, the distance between flight lines is unaffected.
*   **Implication:** Higher overlap improves 3D reconstruction quality but increases the number of photos, flight time, and data processing needed.


In [65]:
# Ablation 1
print("\nAblation 1: Increase Overlap to 0.9")
camera_abl1 = copy.deepcopy(camera_x10)
dataset_spec_abl1 = copy.deepcopy(dataset_spec)
dataset_spec_abl1.overlap = 0.9

plan_abl1 = generate_photo_plan_on_grid(camera_abl1, dataset_spec_abl1)
print(f"Generated {len(plan_abl1)} waypoints.")
fig_abl1 = plot_photo_plan(plan_abl1)
fig_abl1.show()



Ablation 1: Increase Overlap to 0.9
Generated 465 waypoints.


### <font color='mediumseagreen'>Ablation 2: Increase Sidelap</font>

*   **Change:** Increase `dataset_spec.sidelap` from the baseline (0.7) to 0.9.
*   **Observation:** The horizontal flight lines are now packed much closer together vertically, increasing the total number of rows (and waypoints). The spacing of waypoints *along* each line remains unchanged.
*   **Reasoning:** Requiring more overlap between adjacent flight lines means the drone must travel a shorter distance sideways to the next line. Since the forward overlap requirement didn't change, the spacing along the flight lines is unaffected.
*   **Implication:** Higher sidelap also boosts reconstruction quality but increases the number of flight lines, total photos, flight time, and data processing.


In [66]:
# Ablation 2
print("\nAblation 2: Increase Sidelap to 0.9")
camera_abl2 = copy.deepcopy(camera_x10)
dataset_spec_abl2 = copy.deepcopy(dataset_spec)
dataset_spec_abl2.sidelap = 0.9

plan_abl2 = generate_photo_plan_on_grid(camera_abl2, dataset_spec_abl2)
print(f"Generated {len(plan_abl2)} waypoints.")
fig_abl2 = plot_photo_plan(plan_abl2)
fig_abl2.show()



Ablation 2: Increase Sidelap to 0.9
Generated 451 waypoints.


### <font color='darkorange'>Ablation 3: Increase Height</font>

*   **Change:** Double `dataset_spec.height` from the baseline (30.48m) to 60.96m.
*   **Observation:** The entire flight grid is sparser. Both the horizontal spacing along lines and the vertical spacing between lines have increased. The total number of waypoints is significantly reduced.
*   **Reasoning:** Flying higher increases the ground area covered by each photo (the footprint). With constant overlap/sidelap percentages, the drone can travel further both forward and sideways between photos while still meeting the requirements.
*   **Implication:** Flying higher covers ground faster with fewer photos, saving time and data. However, this comes at the cost of lower ground resolution (less detail) in the images.


In [67]:
# Ablation 3
print("\nAblation 3: Double Height")
camera_abl3 = copy.deepcopy(camera_x10)
dataset_spec_abl3 = copy.deepcopy(dataset_spec)
dataset_spec_abl3.height = dataset_spec.height * 2 # Double the height

plan_abl3 = generate_photo_plan_on_grid(camera_abl3, dataset_spec_abl3)
print(f"Generated {len(plan_abl3)} waypoints.")
fig_abl3 = plot_photo_plan(plan_abl3)
fig_abl3.show()




Ablation 3: Double Height
Generated 48 waypoints.


### <font color='crimson'>Ablation 4: Increase Exposure Time</font>

*   **Change:** Increase `dataset_spec.exposure_time_ms` from the baseline (2ms) to 1000ms.
*   **Observation:** The plotted flight path (waypoint positions and density) appears identical to the baseline plan. However, the "Capture Speed" value displayed in the plot title has decreased dramatically.
*   **Reasoning:** Exposure time dictates how long the camera shutter is open. It directly influences the maximum speed allowed *during* capture to avoid motion blur but does not affect the geometric layout (waypoint positions) needed to achieve the desired ground coverage and overlap.
*   **Implication:** Longer exposure times (needed in low light) force the drone to fly much slower *at the moment of capture*, even if the path itself doesn't change. This can significantly increase the total mission duration compared to flying the same path in bright conditions with short exposures.


In [68]:
# Ablation 4
print("\nAblation 4: Increase Exposure Time to 1000ms")
camera_abl4 = copy.deepcopy(camera_x10)
dataset_spec_abl4 = copy.deepcopy(dataset_spec)
dataset_spec_abl4.exposure_time_ms = 1000 # Drastically increase exposure time

plan_abl4 = generate_photo_plan_on_grid(camera_abl4, dataset_spec_abl4)
print(f"Generated {len(plan_abl4)} waypoints.")
fig_abl4 = plot_photo_plan(plan_abl4)
fig_abl4.show()



Ablation 4: Increase Exposure Time to 1000ms
Generated 165 waypoints.


### <font color='rebeccapurple'>Ablation 5: Increase Focal Length (Simulated Zoom)</font>

*   **Change:** Double the camera's focal length in pixels (`fx`, `fy`) in the copied `camera_` object. Keep `dataset_spec` at baseline.
*   **Observation:** The flight grid becomes much denser in both X and Y directions. The spacing between waypoints along lines and between lines decreases significantly, resulting in a much higher total waypoint count.
*   **Reasoning:** A higher focal length (in pixels, with sensor/image size constant) corresponds to a more "zoomed-in" view, meaning each photo captures a smaller area on the ground (smaller footprint). To maintain 


In [72]:
# Ablation 5 Code
print("\nAblation 5: Double Focal Length (fx, fy)")
camera_abl5 = copy.deepcopy(camera_x10)
dataset_spec_abl5 = copy.deepcopy(dataset_spec)
camera_abl5.fx = camera_x10.fx * 2.0 # Double fx
camera_abl5.fy = camera_x10.fy * 2.0 # Double fy
print(f"Modified Camera fx={camera_abl5.fx:.2f}, fy={camera_abl5.fy:.2f}")

plan_abl5 = generate_photo_plan_on_grid(camera_abl5, dataset_spec_abl5)
print(f"Generated {len(plan_abl5)} waypoints.")
fig_abl5 = plot_photo_plan(plan_abl5)
fig_abl5.show()



Ablation 5: Double Focal Length (fx, fy)
Modified Camera fx=9877.12, fy=9872.98
Generated 588 waypoints.


### <font color='sienna'>Ablation 6: Minimal Overlap & Sidelap</font>

*   **Change:** Reduce `dataset_spec.overlap` and `dataset_spec.sidelap` to a minimal value (0.01 , 1%). Keep other parameters at baseline.
*   **Observation:** The flight grid becomes extremely sparse. Both horizontal and vertical distances between waypoints are maximized, approaching the full size of the image footprint. Very few waypoints are generated.
*   **Reasoning:** With very little required overlap, the drone can travel almost the full width/height of the area captured in one photo before needing to take the next one. This maximizes the spacing between waypoints.
*   **Implication:** While drastically reducing photo count and flight time, such minimal overlap is insufficient for reliable map stitching or 3D model generation in photogrammetry. It demonstrates the lower limit of coverage but highlights why higher overlaps are standard practice for quality results.


In [ ]:
# Ablation 6 Code
print("\nAblation 6: Minimal Overlap/Sidelap (1%)")
camera_abl6 = copy.deepcopy(camera_x10)
dataset_spec_abl6 = copy.deepcopy(dataset_spec)
dataset_spec_abl6.overlap = 0.01 # Set overlap to 1%
dataset_spec_abl6.sidelap = 0.01 # Set sidelap to 1%

plan_abl6 = generate_photo_plan_on_grid(camera_abl6, dataset_spec_abl6)
print(f"Generated {len(plan_abl6)} waypoints.")
fig_abl6 = plot_photo_plan(plan_abl6)
fig_abl6.show()



Ablation 6: Minimal Overlap/Sidelap (5%)
Generated 20 waypoints.


### <font color='aqua'>Key Takeaways | Learning Outcomes</font>

This project brought together camera modeling, flight planning, and data visualization for drone-based mapping. These are the main insights I gained:

*   **Camera & Photogrammetry Basics:** I started with very little knowledge of camera optics or drone surveying. Building the pinhole camera model in code helped me understand how focal length and sensor size determine the ground footprint of each image, and why overlap between photos is essential for stitching and 3D reconstruction.

*   **Visual Feedback with Plotly:** Plotting the flight paths made the concepts click.Seeing how adjusting height or overlap changed the density and layout, reinforced the math behind it.

*   **New Tools & Workflows:** This was my first project using:
    *   **Jupyter Notebooks** for combining explanations, code, and plots.
    *   **Plotly** for interactive, customizable visualizations.
    *   **Git branches and Pull Requests** to manage development and code review.

Overall, assembling the planner step by step and testing different parameters gave me practical experience in translating mathematical concepts into a working system—and in communicating results effectively through visualization.